In [ ]:
from aim_perception import pipeline
from aim_perception.evaluation import InferenceEvaluation

In [ ]:
abs_path = '/home/ubuntu/aim/vehicle_dataset'
save_path = '/home/ubuntu/aim/aim-perception/checkpoints/resnet_18/model.pt'
wandb_project = 'final_model'
wandb_run = 'resnet_18'
epochs = 40
fine_tune_epochs=10
image_size = 96
batch_size = 256
weight_decay = 1e-4
model_name = 'resnet_18'
model_kwargs = dict(dropout=0.05)
swa_kwargs = dict(swa_start=30, swa_freq=1)

wandb_config_updates = dict(
    image_size=image_size,
    batch_size=batch_size,
    weight_decay=weight_decay,
    resnet_size=18,
    dropout=0.05,
    swa_kwargs=swa_kwargs,
)

In [ ]:
# Load data loaders
train_loader, val_loader, test_loader = pipeline.create_data_loaders(root_data_path=abs_path, batch_size=batch_size, image_size=(image_size, image_size))

# Load model and optimizer
torch_model, optimizer = pipeline.create_model_and_optimimzer(model_name, model_kwargs, weight_decay=weight_decay, swa_kwargs=swa_kwargs)

# Train model
model = pipeline.train_model(
    model=torch_model,
    optimizer=optimizer, 
    train_loader=train_loader,
    val_loader=val_loader,
    wandb_project=wandb_project,
    wandb_run_name=wandb_run,
    epochs=epochs,
    save_path=save_path,
    fine_tune_epochs=fine_tune_epochs,
    wandb_config_updates=wandb_config_updates
)

# Run inference on test
test_eval = InferenceEvaluation(model, test_loader)
test_eval.classification_report

Train percent: 75.00150253026095
Val percent: 14.99885807700167
Test percent: 9.99963939273737


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fagundo. Use `wandb login --relogin` to force relogin


Moving model to cuda...
[1,   101] train loss: 2.119 | val loss: 2.043 | val bal-acc: 0.445


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/aim/aim-perception/checkpoints/resnet_18/model.pt'